In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

In [3]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
x_train = series[:7000, :n_steps]
x_valid = series[7000:9000, :n_steps]
x_test = series[9000:, :n_steps]

y = np.empty((10000, n_steps, 10))

for step_ahead in range(1, 10 + 1):
    y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]

y_train = y[:7000]
y_valid = y[7000:9000]
y_test = y[9000:]

In [4]:
def last_time_step_mse(y_true, y_pred):
    return keras.metrics.mean_squared_error(y_true[:, -1], y_pred[:, -1])

Deep RNN with Batch Norm

In [5]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, input_shape=[None, 1], 
                           return_sequences=True),
    keras.layers.BatchNormalization(),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.BatchNormalization(),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss='mse', 
              optimizer='adam', 
              metrics=[last_time_step_mse])

history = model.fit(x_train, y_train, epochs=20, 
                    validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 33s 145ms/step - loss: 0.1929 - last_time_step_mse: 0.1902 - val_loss: 0.0877 - val_last_time_step_mse: 0.0832
Epoch 2/20
219/219 [==============================] - 31s 142ms/step - loss: 0.0537 - last_time_step_mse: 0.0449 - val_loss: 0.0549 - val_last_time_step_mse: 0.0462
Epoch 3/20
219/219 [==============================] - 30s 139ms/step - loss: 0.0471 - last_time_step_mse: 0.0375 - val_loss: 0.0451 - val_last_time_step_mse: 0.0358
Epoch 4/20
219/219 [==============================] - 28s 127ms/step - loss: 0.0437 - last_time_step_mse: 0.0337 - val_loss: 0.0418 - val_last_time_step_mse: 0.0314
Epoch 5/20
219/219 [==============================] - 29s 134ms/step - loss: 0.0409 - last_time_step_mse: 0.0306 - val_loss: 0.0391 - val_last_time_step_mse: 0.0287
Epoch 6/20
219/219 [==============================] - 29s 134ms/step - loss: 0.0385 - last_time_step_mse: 0.0275 - val_loss: 0.0379 - val_last_time_step_mse: 0.0273
Epoch 7/20

Deep RNNs with Layer Norm

In [6]:
from tensorflow.keras.layers import LayerNormalization

class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation='tanh', **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units, 
                                                          activation=None)
        self.layer_norm = LayerNormalization()
        self.activation = keras.activations.get(activation)
        
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        if inputs is not None:
            batch_size = tf.shape(inputs)[0]
            dtype = inputs.dtype
        return [tf.zeros([batch_size, self.state_size], dtype=dtype)]
    
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [7]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), input_shape=[None, 1], 
                           return_sequences=True),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss='mse', 
              optimizer='adam', 
              metrics=[last_time_step_mse])

history = model.fit(x_train, y_train, epochs=20, 
                    validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 67s 294ms/step - loss: 0.1594 - last_time_step_mse: 0.1521 - val_loss: 0.0735 - val_last_time_step_mse: 0.0624
Epoch 2/20
219/219 [==============================] - 64s 294ms/step - loss: 0.0635 - last_time_step_mse: 0.0508 - val_loss: 0.0561 - val_last_time_step_mse: 0.0415
Epoch 3/20
219/219 [==============================] - 64s 291ms/step - loss: 0.0527 - last_time_step_mse: 0.0364 - val_loss: 0.0498 - val_last_time_step_mse: 0.0328
Epoch 4/20
219/219 [==============================] - 64s 293ms/step - loss: 0.0466 - last_time_step_mse: 0.0307 - val_loss: 0.0440 - val_last_time_step_mse: 0.0288
Epoch 5/20
219/219 [==============================] - 64s 294ms/step - loss: 0.0423 - last_time_step_mse: 0.0276 - val_loss: 0.0410 - val_last_time_step_mse: 0.0252
Epoch 6/20
219/219 [==============================] - 65s 295ms/step - loss: 0.0389 - last_time_step_mse: 0.0241 - val_loss: 0.0373 - val_last_time_step_mse: 0.0218
Epoch 7/20

Creating a Custom RNN Class

In [8]:
class MyRNN(keras.layers.Layer):
    def __init__(self, cell, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences
        self.get_initial_state = getattr(
            self.cell, 'get_initial_state', self.fallback_initial_state)
        
    def fallback_initial_state(self, inputs):
        batch_size = tf.shape(inputs)[0]
        return [tf.zeros([batch_size, self.cell.state_size], dtype=inputs.dtype)]
    
    @tf.function
    def call(self, inputs):
        states = self.get_initial_state(inputs)
        shape = tf.shape(inputs)
        batch_size = shape[0]
        n_steps = shape[1]
        sequences = tf.TensorArray(
            inputs.dtype, size=(n_steps if self.return_sequences else 0))
        outputs = tf.zeros(shape=[batch_size, self.cell.output_size], dtype=inputs.dtype)
        for step in tf.range(n_steps):
            outputs, states = self.cell(inputs[:, step], states)
            if self.return_sequences:
                sequences = sequences.write(step, outputs)
        if self.return_sequences:
            return tf.transpose(sequences.stack(), [1, 0, 2])
        else:
            return outputs

In [9]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    MyRNN(LNSimpleRNNCell(20), input_shape=[None, 1], 
                           return_sequences=True),
    MyRNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss='mse', 
              optimizer='adam', 
              metrics=[last_time_step_mse])

history = model.fit(x_train, y_train, epochs=20, 
                    validation_data=(x_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 67s 303ms/step - loss: 0.1597 - last_time_step_mse: 0.1569 - val_loss: 0.0743 - val_last_time_step_mse: 0.0717
Epoch 2/20
219/219 [==============================] - 65s 297ms/step - loss: 0.0643 - last_time_step_mse: 0.0538 - val_loss: 0.0573 - val_last_time_step_mse: 0.0427
Epoch 3/20
219/219 [==============================] - 66s 299ms/step - loss: 0.0535 - last_time_step_mse: 0.0396 - val_loss: 0.0504 - val_last_time_step_mse: 0.0338
Epoch 4/20
219/219 [==============================] - 65s 297ms/step - loss: 0.0468 - last_time_step_mse: 0.0313 - val_loss: 0.0442 - val_last_time_step_mse: 0.0306
Epoch 5/20
219/219 [==============================] - 63s 288ms/step - loss: 0.0423 - last_time_step_mse: 0.0279 - val_loss: 0.0409 - val_last_time_step_mse: 0.0260
Epoch 6/20
219/219 [==============================] - 63s 289ms/step - loss: 0.0389 - last_time_step_mse: 0.0244 - val_loss: 0.0368 - val_last_time_step_mse: 0.0217
Epoch 7/20